In [3]:
from pymongo import MongoClient

In [4]:
client = MongoClient()

In [5]:
nvd_database = client['vulnerability']['nvd']

In [6]:
nvd_database.find_one({}).keys()

dict_keys(['_id', 'cve', 'configurations', 'impact', 'publishedDate', 'lastModifiedDate'])

In [7]:
from collections import defaultdict
d = defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(
                lambda: defaultdict(set))))

In [70]:
from itertools import chain
from pprint import pprint

results = nvd_database.find({}, {
    'cve.CVE_data_meta.ID': 1,
    'configurations': 1,
    '_id': 0,
})

def extract_cpe_matches(node):
    if 'cpe_match' in node:
        return node['cpe_match']
    if 'children' in node:
        return chain(*[extract_cpe_matches(child) for child in node['children']])
    # pprint(node)
    return []

def yield_cpe_match(results):
    for r in results:
        cve_id = r['cve']['CVE_data_meta']['ID']
        for node in r['configurations']['nodes']:
            for cpe_match in extract_cpe_matches(node):
                yield cpe_match

keys = set()
for cpe_match in yield_cpe_match(results):
    for key in cpe_match.keys():
        if key not in keys:
            print(key)
            keys.add(key)
    '''
    component_type = 
    vendor_name = 
    product_name =
    product_version_tuple =
    d[component_type][vendor_name][product_name][product_version_tuple].append(cve.update(ids))
    '''

vulnerable
cpe23Uri
versionEndIncluding
versionStartIncluding
versionEndExcluding
versionStartExcluding


In [71]:
cpe_uri = cpe_match['cpe23Uri']
cpe_uri

'cpe:2.3:a:webmproject:libwebm:*:*:*:*:*:*:*:*'

In [72]:
component_type, vendor_name, product_name, product_version = cpe_uri.split(':')[2:6]

In [73]:
component_type

'a'

In [74]:
vendor_name

'webmproject'

In [75]:
product_name

'libwebm'

In [76]:
product_version

'*'

In [77]:
d = defaultdict(
    lambda: defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(set))))


In [78]:
d['a']['b']['c']['d'].add(1)

In [79]:
d['o']['b']['c']['d'].add(1)

In [80]:
d.keys()

dict_keys(['a', 'o'])

In [81]:
from collections import defaultdict
from itertools import chain

def extract_cpe_matches(node):
    if 'cpe_match' in node:
        return node['cpe_match']
    if 'children' in node:
        return chain(*[extract_cpe_matches(child) for child in node['children']])
    return []

def yield_cpe_pack(results):
    for r in results:
        cve_id = r['cve']['CVE_data_meta']['ID']
        for node in r['configurations']['nodes']:
            for cpe_match in extract_cpe_matches(node):
                yield cve_id, cpe_match

In [82]:
results = nvd_database.find({}, {
    'cve.CVE_data_meta.ID': 1,
    'configurations': 1,
    '_id': 0,
})
component_types = set()

d = defaultdict(
    lambda: defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(set))))

for cve_id, cpe_match in yield_cpe_pack(results):
    component_type, vendor_name, product_name, product_version = cpe_match['cpe23Uri'].split(':')[2:6]
    if product_version:
        version_a_including = product_version
        version_b_including = product_version
    version_a_excluding = cpe_match['versionStartExcluding'] if 'versionStartExcluding' in cpe_match else None
    version_a_including = cpe_match['versionStartIncluding'] if 'versionStartIncluding' in cpe_match else None
    version_b_including = cpe_match['versionEndIncluding'] if 'versionEndIncluding' in cpe_match else None
    version_b_excluding = cpe_match['versionEndExcluding'] if 'versionEndExcluding' in cpe_match else None
    product_version_pack = (
        version_a_excluding,
        version_a_including,
        version_b_including,
        version_b_excluding)
    component_types.add(component_type)
    d[component_type][vendor_name][product_name][product_version_pack].add(cve_id)

In [83]:
component_types

{'a', 'h', 'o'}

In [84]:
nvd_database.count_documents({})

119564

In [85]:
results = nvd_database.find({}, {
    'cve.CVE_data_meta.ID': 1,
    'configurations': 1,
    '_id': 0,
})
for i, r in enumerate(results):
    if i == 3:
        break

In [86]:
r

{'configurations': {'CVE_data_version': '4.0',
  'nodes': [{'cpe_match': [{'cpe23Uri': 'cpe:2.3:a:hp:dtmail:*:*:*:*:*:*:*:*',
      'vulnerable': True},
     {'cpe23Uri': 'cpe:2.3:a:university_of_washington:pine:4.02:*:*:*:*:*:*:*',
      'vulnerable': True}],
    'operator': 'OR'},
   {'cpe_match': [{'cpe23Uri': 'cpe:2.3:o:sco:unixware:7.0:*:*:*:*:*:*:*',
      'vulnerable': True}],
    'operator': 'OR'}]},
 'cve': {'CVE_data_meta': {'ID': 'CVE-1999-0004'}}}

In [87]:
list(yield_cpe_pack([r]))

[('CVE-1999-0004',
  {'cpe23Uri': 'cpe:2.3:a:hp:dtmail:*:*:*:*:*:*:*:*', 'vulnerable': True}),
 ('CVE-1999-0004',
  {'cpe23Uri': 'cpe:2.3:a:university_of_washington:pine:4.02:*:*:*:*:*:*:*',
   'vulnerable': True}),
 ('CVE-1999-0004',
  {'cpe23Uri': 'cpe:2.3:o:sco:unixware:7.0:*:*:*:*:*:*:*',
   'vulnerable': True})]

In [91]:
for cve_id, cpe_match in yield_cpe_pack([r]):
    component_type, vendor_name, product_name, product_version = cpe_match['cpe23Uri'].split(':')[2:6]
    print(component_type)

a
a
o


In [94]:
len(d['a'])

19248

In [95]:
len(d['h'])

918

In [96]:
len(d['o'])

788

In [99]:
'huawei' in sorted(d['h'].keys())

True